In [ ]:
#HOMEWORK #1 CS 260
#AMIR DHILLON 
#January 25 2021
#UID : 005-649-833
# References :
#https://github.com/AnupamMicrosoft/PyTorch-Classification/blob/master/README.md
#https://pytorch.org/get-started/locally/
#https://stackoverflow.com/questions/61599465/why-does-by-torch-optim-sgd-method-learning-rate-change
#https://stackoverflow.com/questions/48324152/pytorch-how-to-change-the-learning-rate-of-an-optimizer-at-any-given-moment-no/48324389

In [29]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn import functional as F
import matplotlib.pyplot as plt

In [31]:
#Hyper-parameters 

batch_size = 64 #this is not changed
num_epochs = 10  # number of times you will iterate through the full training data
learning_rate = 0.0001 ## step size used by SGD 
momentum = 0.9 ## Momentum is a moving average of our gradients (helps to keep direction)

In [32]:
## USE THIS SNIPPET TO GET BINARY TRAIN/TEST DATA

train_data = datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
test_data = datasets.MNIST('./data', train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

subset_indices = ((train_data.targets == 0) + (train_data.targets == 1)).nonzero()
train_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size, 
  shuffle=False,sampler=SubsetRandomSampler(subset_indices.view(-1)))


subset_indices = ((test_data.targets == 0) + (test_data.targets == 1)).nonzero()
test_loader = torch.utils.data.DataLoader(test_data,batch_size=batch_size, 
  shuffle=False,sampler=SubsetRandomSampler(subset_indices.view(-1)))


In [37]:
#(image size, number of classes)
#Logistic regression model and Loss

Logistics_model = nn.Linear(28*28,1)
Logistics_momentum = nn.Linear(28*28,1)

#Logistic Regression Optimizer

optimizer_logistics_reg = torch.optim.SGD(Logistics_model.parameters(), lr=learning_rate)
optimizer_logistics_momentum = torch.optim.SGD(Logistics_momentum.parameters(), lr=learning_rate, momentum=momentum)

#SVM/regression  
#(image size, number of classes)
SVM_model = nn.Linear(28*28,1)
SVM_Momentum = nn.Linear(28*28,1)


optimizer_SGD= torch.optim.SGD(SVM_model.parameters(),lr=learning_rate)
optimizer_Momentum= torch.optim.SGD(SVM_Momentum.parameters(),lr=lrate,momentum=momentum)




In [48]:
# Training the Model


for epoch in range(num_epochs):
    total_loss = 0
    total_batches=0
    
    total_loss_momentum=0
    total_loss_logression=0
    total_loss_logression_momentum=0
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 28*28)
        #Convert labels from 0,1 to -1,1
        labels = 2*(labels.float()-0.5)
        label_momentum = 2*(labels.float()-0.5)
        
        ## TODO 
        output = SVM_model(images)
        output_momentum = SVM_Momentum(images)
        output_logression=Logistics_model(images)
        output_logression_momentum=Logistics_momentum(images)
        
        loss_SVM = torch.sum(torch.clamp(1 - output.t()*labels, min=0))/batch_size
        loss_SVM_momentum = torch.sum(torch.clamp(1 - output_momentum.t()*label_momentum, min=0))/batch_size
        loss_logression=torch.sum(torch.log(1 + torch.exp(-(output_logression.t()*labels))))/batch_size
        loss_log_momentum=torch.sum(torch.log(1 + torch.exp(-(output_logression_momentum.t()*labels))))/batch_size
        
        optimizer_SGD.zero_grad()
        loss_SVM.backward()
        optimizer_SGD.step()
        
        optimizer_Momentum.zero_grad()
        loss_SVM_momentum.backward()
        optimizer_Momentum.step()
        
        optimizer_logistics_reg.zero_grad()
        loss_logression.backward()
        optimizer_logistics_reg.step()
        
        optimizer_logistics_momentum.zero_grad()
        loss_log_momentum.backward()
        optimizer_logistics_momentum.step()
        
        total_batches=total_batches+1
        total_loss = total_loss+loss_SVM.item()
        total_loss_momentum=total_loss_momentum+loss_SVM_momentum.item()
        total_loss_logression=total_loss_logression+loss_logression.item()
        total_loss_logression_momentum=total_loss_logression_momentum+loss_log_momentum.item()
    ## Print your results every epoch
    print('Average loss SVM of epoch::{} is : [{}], SVM momentum: [{}]'
          .format(epoch+1,total_loss/total_batches,total_loss_momen/total_batches))
    print('Average loss Logistic Regression of epoch::{} is : [{}], Logistic Regression momentum: [{}]'
          .format(epoch+1,total_loss_logression/total_batches,total_loss_logression_momentum/total_batches))


# Test the Model
correct = 0.
correct_momentum=0.
correct_log=0.
correct_log_momentum=0.
total = 0.
for images, labels in test_loader:
    images = images.view(-1, 28*28)
    
## Put your prediction code here, currently use a random prediction
    outputs = SVM_model(images)
    prediction = outputs.data>=0
    
    output_momentum=SVM_Momentum(images)
    prediction_momentum=output_momentum.data>=0
    
    output_log=torch.sigmoid(Logistics_model(images))
    prediction_log=output_log.data>=0.5
    
    output_logression_momentum=torch.sigmoid(Logistics_momentum(images))
    prediction_log_momentum=output_logression_momentum.data>=0.5

    correct += (prediction.view(-1).long() == labels).sum()
    correct_momentum += (prediction_momentum.view(-1).long() == labels).sum()
    correct_log += (prediction_log.view(-1).long() == labels).sum()
    correct_log_momentum += (prediction_log_momentum.view(-1).long() == labels).sum()
    total += images.shape[0]
print('Accuracy of the model on the test images ------SVM: %f %%' % (100 * (correct.float() / total)))
print('Accuracy of the model on the test images ------SVM/Momentum: %f %%' % (100 * (correct_momentum.float() / total)))
print('Accuracy of the model on the test images ------Logistic RG: %f %%' % (100 * (correct_log.float() / total)))
print('Accuracy of the model on the test images ------Logistic RG/Momentum: %f %%' % (100 * (correct_log_momentum.float() / total)))

Average loss SVM of epoch::1 is : [0.005270114295523275], SVM momentum: [0.0016987147938572998]
Average loss Logistic Regression of epoch::1 is : [0.01528551916808191], Logistic Regression momentum: [0.004845063480774335]
Average loss SVM of epoch::2 is : [0.005234157337308532], SVM momentum: [0.0016987147938572998]
Average loss Logistic Regression of epoch::2 is : [0.015174259034201804], Logistic Regression momentum: [0.004819011560293159]
Average loss SVM of epoch::3 is : [0.005198388407477225], SVM momentum: [0.0016987147938572998]
Average loss Logistic Regression of epoch::3 is : [0.015065179001822165], Logistic Regression momentum: [0.004791478332451949]
Average loss SVM of epoch::4 is : [0.005162965407538594], SVM momentum: [0.0016987147938572998]
Average loss Logistic Regression of epoch::4 is : [0.014958574471882346], Logistic Regression momentum: [0.004764854462906681]
Average loss SVM of epoch::5 is : [0.005127342107395331], SVM momentum: [0.0016987147938572998]
Average loss 